In [1]:
from ray.rllib.agents.ppo import PPOTrainer
import numpy as np

import industrial_benchmarks_train as ibt
import pandas as pd
import matplotlib.pyplot as plt
from ppo.policy import LSTMPPOPolicy

# Create Agent

In [2]:
from envs.IBGym_mod_envs import IBGymModded

ibt.config["num_workers"] = 1
# agent = PPOTrainer(config=ibt.config, env=None)
# agent.restore(
#     "tmp/ray_exp_logs/industrial_benchmark/PPOTrainer_IBGym-v1_247b0_00000_0_2022-10-14_10-06-44/checkpoint_000150/checkpoint-150")
env = IBGymModded(70, reward_type='classic', action_type='discrete', reset_after_timesteps=1000,
                       observation_type='include_past', n_past_timesteps=30)
save_path = 'ppo/lstm_ppo_policy.pkl'
policy = LSTMPPOPolicy.load(save_path)

/home/hamza/PycharmProjects/StateCompression/venv/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


# Test Function

In [3]:
def test(time_steps= 1000):
    inference_data = {"gain": [], "velocity": [], "shift": [], "actions": []}
    episode_reward = 0
    done = False
    obs = env.reset()

    state = []
    use_lstm = True#ibt.config["model"]["use_lstm"]
    use_attention = False#ibt.config["model"]["use_attention"]
    # if use_lstm:
    #     cell_size = ibt.config["model"]["lstm_cell_size"]
    #     state = [np.zeros(cell_size, np.float32),
    #              np.zeros(cell_size, np.float32)]
    # if use_attention:
    #     num_transformers = ibt.config["model"]["attention_num_transformer_units"]
    #     attention_dim = ibt.config["model"]["attention_dim"]
    #     memory = 50  # ibt.config["model"]["attention_memory_inference"]
    #     state = [
    #         np.zeros([memory, attention_dim], np.float32)
    #         for _ in range(num_transformers)
    #     ]
    i=0

    while i < time_steps:
        # inference_data["obs"].extend(obs)
        inference_data["gain"].append(obs[2])
        inference_data["velocity"].append(obs[1])
        inference_data["shift"].append(obs[3])

        action = policy.get_action(obs)

        inference_data['actions'].append(action)

        obs, reward, done, info = env.step(action)
        print(obs)
        print(reward)
        # inference_data["action"].extend(action)
        # inference_data["reward"].append(reward)
        episode_reward += reward

        # if use_attention:
        #     state = [
        #         np.concatenate([state[i], [state_out[i]]], axis=0)[1:]
        #         for i in range(num_transformers)
        #     ]
        i=i+1

    print(f"Average Reward: {episode_reward/time_steps}")
    return inference_data

# Plot Function

In [4]:
def plot_inf_data(inference_data):
    df = pd.DataFrame(inference_data)
    plt.plot(df['actions'])
    plt.legend("actions")
    plt.figure(figsize = (20, 8), dpi = 800)
    plt.plot(df['gain'])
    plt.legend("gain")
    plt.figure(figsize = (20, 8), dpi = 800)
    plt.show()
    plt.plot(df['velocity'])
    plt.legend("velocity")
    plt.figure(figsize = (20, 8), dpi = 800)
    plt.show()
    plt.plot(df['shift'])
    plt.legend("shift")
    plt.figure(figsize = (20, 8), dpi = 800)

    plt.show()

# Create test data and plot

In [ ]:
inference_data = test(1000)

plot_inf_data(inference_data)

2022-10-14 15:28:08,630	WARNING trainer.py:2540 -- You have specified 3 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Trainer.train()`. Instead, you will have to call `Trainer.evaluate()` manually in order to trigger an evaluation run.
